In [ ]:
import os
import requests
from typing import Dict, Any, Optional, List, Union
import pandas as pd
from dotenv import load_dotenv

env_path = r"<Path>\Auth.env"
load_dotenv(dotenv_path = env_path)

# Set Tableau Server / Site
TABLEAU_SERVER = os.getenv("TABLEAU_SERVER") # Cloud URL
API_VERSION = os.getenv("API_VERSION") # https://help.tableau.com/current/api/rest_api/en-us/REST/rest_api_concepts_versions.htm
SITE_CONTENT_URL = os.getenv("SITE_CONTENT_URL") # Site Content URL

PAT_NAME = os.getenv("token_name")
PAT_SECRET = os.getenv("cloud_token_pw")

print("SERVER:", TABLEAU_SERVER)
print("API_VERSION:", API_VERSION)
print("SITE_CONTENT_URL:", SITE_CONTENT_URL)
print("PAT_NAME:", PAT_NAME)

In [ ]:
signin_url = f"{TABLEAU_SERVER}/api/{API_VERSION}/auth/signin"

# Request Body is "XML" Format
ts_request = f"""
<tsRequest xmlns="http://tableau.com/api">
    <credentials personalAccessTokenName="{PAT_NAME}"
            personalAccessTokenSecret="{PAT_SECRET}">
    <site contentUrl="{SITE_CONTENT_URL}" />
    </credentials>
</tsRequest>
""".strip()

headers = {
    "Content-Type": "application/xml",
    "Accept": "application/json",  # Request JSON here
    }

resp = requests.post(signin_url, data=ts_request.encode("utf-8"), headers=headers)

if not resp.ok:
        raise RuntimeError(f"Fail to Sign in : {resp.status_code}\n{resp.text[:800]}")

data = resp.json()

# Checking JSON Structure
# Extract credentials / site / user
creds = data.get("credentials", {})
AUTH_TOKEN = creds.get("token")

site = creds.get("site", {})
user = creds.get("user", {})

SITE_ID = site.get("id")
USER_ID = user.get("id")